In [1]:
# python native
import os
import json
import random
import datetime
from functools import partial

# external library
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
import albumentations as A

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models

# visualization
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
IMAGE_ROOT = "/data/ephemeral/home/level2-cv-semanticsegmentation-cv-06-lv3/data/fold_test/images"
#LABEL_ROOT = "/data/ephemeral/home/level2-cv-semanticsegmentation-cv-06-lv3/data/fold_0/annos"

In [3]:
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]

In [4]:
CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
IND2CLASS = {v: k for k, v in CLASS2IND.items()}


In [7]:
pngs = {
    os.path.relpath(os.path.join(root, fname), start=IMAGE_ROOT)
    for root, _dirs, files in os.walk(IMAGE_ROOT)
    for fname in files
    if os.path.splitext(fname)[1].lower() == ".png"
}

In [8]:
print("Sample PNG paths:", list(pngs)[:5])

Sample PNG paths: ['ID047_image1661389291522.png', 'ID270_image1664154396535.png', 'ID257_image1663724282043.png', 'ID221_image1663120209081.png', 'ID220_image1663120015357.png']


In [9]:
len(pngs)

288

In [10]:
pngs = sorted(pngs)

In [11]:
df_input = pd.read_csv('./output_9750.csv')

In [12]:
pngs[0].split('_')[1]

'image1661319116107.png'

In [13]:
image_dict = {name.split("_")[1]:name for name in list(pngs)}


In [14]:
df_input["image_path"] = df_input['image_name'].apply(lambda x: image_dict[x])

In [15]:
def rle_to_mask(rle, height, width):
    mask = np.zeros(height * width, dtype=np.uint8)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2] - 1
    lengths = array[1::2]
    for start, length in zip(starts, lengths):
        mask[start:start + length] = 1
    return mask.reshape((height, width))

In [16]:
class XRayDataset(Dataset):
    def __init__(self, df, is_train=True, transforms=None):
        
        
        self.df = df
        _filenames = self.df['image_name'].unique()

        self.filenames = _filenames
        self.is_train = is_train
        self.transforms = transforms
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, item):
        # image_name = self.filenames[item]
        # image_path = os.path.join(IMAGE_ROOT, image_name)

        df_ = self.df[self.df['image_name']==self.filenames[item]]

        image_path = df_['image_path'].iloc[0]

        # image = cv2.imread(image_path)
        image = cv2.imread(os.path.join(IMAGE_ROOT, image_path))
        # image = image / 255.

        label_shape = tuple(image.shape[:2]) + (len(CLASSES), )
        label = np.zeros(label_shape, dtype=np.uint8)

        for index, row in df_.iterrows():
            rle = row['rle']
            class_ind = CLASS2IND[row['class']]
            mask = rle_to_mask(rle, 2048, 2048)
            label[..., class_ind] = mask

                
            
        return image, label, image_path

In [17]:
train_dataset = XRayDataset(df_input,is_train=True, transforms=None)

In [21]:
train_dataset[0][2]

'ID040_image1661319116107.png'

In [23]:
SAVE_ROOT = './crop_test_Trapezoid'

In [20]:
from tqdm import tqdm 

In [22]:
CLASS2IND['Trapezoid']

20

In [24]:
fold_dir = os.path.join(SAVE_ROOT)
os.makedirs(fold_dir, exist_ok=True)

image_dir = os.path.join(fold_dir, 'images')
os.makedirs(image_dir, exist_ok=True)

# annos_dir = os.path.join(fold_dir, 'annos')
# os.makedirs(annos_dir, exist_ok=True)

crop_size = (224, 224)  # (height, width)

for i in tqdm(range(len(train_dataset))):
    image, label, image_path = train_dataset[i]

    # for k in [CLASS2IND['Trapezoid']]: # ,CLASS2IND['Pisiform']
    # for k in [CLASS2IND['Pisiform']]:
    for k in [CLASS2IND['Trapezoid']]:

        coords = np.column_stack(np.where(label[:,:,k] > 0))

        x_min, y_min = coords.min(axis=0)  
        x_max, y_max = coords.max(axis=0)  


        if abs(x_max - x_min) > crop_size[0] or abs(y_max - y_min) > crop_size[1]:
            print(abs(x_max - x_min))
            print(abs(y_max - y_min))
            print('????')


        center_x = (x_min + x_max) // 2
        center_y = (y_min + y_max) // 2

        half_h = crop_size[0] // 2
        half_w = crop_size[1] // 2

        # cropped_image = image[x_min-10:x_max+10,y_min-10:y_max+10]
        # cropped_mask = label[x_min-10:x_max+10,y_min-10:y_max+10,k]

        start_x = max(center_x - half_h, 0)
        start_y = max(center_y - half_w, 0)
        end_x = start_x + crop_size[0]
        end_y = start_y + crop_size[1]

        if end_x > image.shape[0]:
            start_x = image.shape[0] - crop_size[0]
            end_x = image.shape[0]

        if end_y > image.shape[1]:
            start_y = image.shape[1] - crop_size[1]
            end_y = image.shape[1]

        cropped_image = image[start_x:end_x, start_y:end_y]
        # cropped_mask = label[start_x:end_x, start_y:end_y, k]

        output_image_path = os.path.join(image_dir,str(k)+'_'+image_path)

    # break 
        cv2.imwrite(output_image_path, cropped_image)

        # t = str(k)+'_'+label_path.split('/')[-1]
        # t = t[:-4] + "png"
        # output_label_path = os.path.join(annos_dir,t)

        # cv2.imwrite(output_label_path, cropped_mask)


100%|██████████| 288/288 [03:30<00:00,  1.36it/s]
